 ![CellphoneDB Logo](https://www.cellphonedb.org/images/cellphonedb_logo_33.png) | CellphoneDB is a publicly available repository of curated receptors, ligands and their interactions.

In [ ]:
import pandas as pd
import sys
import os

Checking that environment contains a Python >= 3.8 as required by CellphoneDB.

In [ ]:
print(sys.version)

___
### Input files
The statistial method accepts 5 input files (3 mandatory).
- **cpdb_file_path**: (mandatory) path to the database `cellphonedb.zip`.
- **meta_file_path**: (mandatory) path to the meta file linking cell barcodes to cluster labels `metadata.tsv`.
- **counts_file_path**: (mandatory) paths to normalized counts file (not z-transformed), either in text format or h5ad (recommended) `normalised_log_counts.h5ad`.
- **microenvs_file_path** (optional) path to microenvironment file that groups cell clusters by microenvironments. When providing a microenvironment file, CellphoneDB will restrict the interactions to those cells within a microenvironment.
- **active_tf_path**: (optional) to the active transcription factors.

The `microenvs_file_path` content will depend on the biological question that the researcher wants to answer.

> In this **example** we are studying how cell-cell interactions change between a subset of immune cells and trophoblast cells as the trophoblast differentiate and invade the maternal uterus. This module will randomly permute the cluster labels of all cells whitin each microenvironement (`microenvs_file_path`), 1,000 times (default), to test whether the mean average receptor expression level in a cluster and the average ligand expression level between the interacting clusters is higher than those of the rest cell pairs. This procedure generates a P-value for the likelihood of cell-type specificity of a given receptor–ligand complex.

In [ ]:
cpdb_file_path = r"C:\Users\juanp\Desktop\cpdb_dbv2\v5.0.0\cellphonedb.zip"
meta_file_path = r"C:\Users\juanp\Desktop\CellphoneDB_CM_Other\CMEC_meta.tsv"
counts_file_path = r"C:\Users\juanp\Desktop\CellphoneDB_CM_Other\CMEC_counts.h5ad"
#microenvs_file_path = 'data/microenvironment.tsv'
out_path = r"C:\Users\juanp\Desktop\CellphoneDB_Results_CM_Other\CMEC"
#active_tf_path = 'data/active_TFs.tsv'
#out_path = 'results/method2_withScore'

### Inspect input files

<span style="color:green">**1)**</span> The **metadata** file is compossed of two columns:
- **barcode_sample**: this column indicates the barcode of each cell in the experiment.
- **cell_type**: this column denotes the cell label assigned.

In [ ]:
metadata = pd.read_csv(meta_file_path, sep = '\t')
metadata.head(3)

In [ ]:
import anndata

adata = anndata.read_h5ad(counts_file_path)
adata.shape

<span style="color:green">**2)**</span>  The **counts** files is a scanpy h5ad object. The dimensions and order of this object must coincide with the dimensions of the metadata file (i.e. must have the same number of cells in both files).

Check barcodes in metadata and counts are the same.

In [ ]:
list(adata.obs.index).sort() == list(metadata['barcode']).sort()

<span style="color:green">**3)**</span> **Micronevironments** defines the cell types that belong to a a given microenvironment. CellphoneDB will only calculate interactions between cells that belong to a given microenvironment. In this file we are defining just one microenvionment.

In [ ]:
#microenv = pd.read_csv(microenvs_file_path,
                       #sep = '\t')
#microenv.head(3)

Displaying cells grouped per microenvironment

In [ ]:
#microenv.groupby('microenvironment', group_keys = False)['cell_type'].apply(lambda x : list(x.value_counts().index))

<span style="color:green">**4)**</span> **Active transcription factors** defines trancription factors active in a given cell type.

In [ ]:
#pd.read_csv(active_tf_path, sep = '\t') \
 #   .head(3)

____
### Run statistical analysis
The output of this method will be saved in `output_path` and also returned to the predefined variables.

> The statisical method allows the user to downsample the data with the aim of speeding up the results (subsampling arguments). To this end, CellphoneDB employs a geometric sketching procedure (Hie *et al.* 2019) to preserve the structure of the data without losing information from lowly represented cells. For this tutorial, we have opted to manually downsample the count matrix and the metadata file accordingly.

In [ ]:
from cellphonedb.src.core.methods import cpdb_statistical_analysis_method

cpdb_results = cpdb_statistical_analysis_method.call(
    cpdb_file_path = cpdb_file_path,                 # mandatory: CellphoneDB database zip file.
    meta_file_path = meta_file_path,                 # mandatory: tsv file defining barcodes to cell label.
    counts_file_path = counts_file_path,             # mandatory: normalized count matrix - a path to the counts file, or an in-memory AnnData object
    counts_data = 'hgnc_symbol',                     # defines the gene annotation in counts matrix.
   # active_tfs_file_path = active_tf_path,           # optional: defines cell types and their active TFs.
    #microenvs_file_path = microenvs_file_path,       # optional (default: None): defines cells per microenvironment.
    score_interactions = True,                       # optional: whether to score interactions or not. 
    iterations = 1000,                               # denotes the number of shufflings performed in the analysis.
    threshold = 0.1,                                 # defines the min % of cells expressing a gene for this to be employed in the analysis.
    threads = 5,                                     # number of threads to use in the analysis.
    debug_seed = 42,                                 # debug randome seed. To disable >=0.
    result_precision = 3,                            # Sets the rounding for the mean values in significan_means.
    pvalue = 0.05,                                   # P-value threshold to employ for significance.
    subsampling = False,                             # To enable subsampling the data (geometri sketching).
    subsampling_log = False,                         # (mandatory) enable subsampling log1p for non log-transformed data inputs.
    subsampling_num_pc = 100,                        # Number of componets to subsample via geometric skectching (dafault: 100).
    subsampling_num_cells = 1000,                    # Number of cells to subsample (integer) (default: 1/3 of the dataset).
    separator = '|',                                 # Sets the string to employ to separate cells in the results dataframes "cellA|CellB".
    debug = False,                                   # Saves all intermediate tables employed during the analysis in pkl format.
    output_path = out_path,                          # Path to save results.
    output_suffix = None                             # Replaces the timestamp in the output files by a user defined string in the  (default: None).
    )

Results are save into the predefined file and as a dictionary in the `cpdb_results` variable.

In [ ]:
list(cpdb_results.keys())

___
### Description of output files
Most output files share common columns:
- **id_cp_interaction**: Unique CellphoneDB identifier for each interaction stored in the database.
- **interacting_pair**: Name of the interacting pairs separated by “|”.
- **partner A or B**: Identifier for the first interacting partner (A) or the second (B). It could be: UniProt (prefix simple:) or complex (prefix complex:)
- **gene A or B**: Gene identifier for the first interacting partner (A) or the second (B). The identifier will depend on the input user list.
- **secreted**: True if one of the partners is secreted.
- **Receptor A or B**: True if the first interacting partner (A) or the second (B) is annotated as a receptor in our database.
- **annotation_strategy**: Curated if the interaction was annotated by the CellphoneDB developers. Otherwise, the name of the database where the interaction has been downloaded from.
- **is_integrin**: True if one of the partners is integrin.
- **directionality**: Indiicates the directionality of the interaction and the charactersitics of the interactors.
- **classification**: Pathway classification for the interacting partners.

**Pvalues** fields:
- **cell_a|cell_b**: The p-value resulting from the statistical analysis.

In [ ]:
cpdb_results['pvalues'].head(20)

**Means** fields:

- **means**: Mean values for all the interacting partners: mean value refers to the total mean of the individual partner average expression values in the corresponding interacting pairs of cell types. If one of the mean values is 0, then the total mean is set to 0.

In [ ]:
cpdb_results['means'].head(2)

**Significant means** fields:
- **significant_mean**: Significant mean calculation for all the interacting partners. If the interaction has been found relevant, the value will be the mean. Alternatively, the value is absent.

In [ ]:
cpdb_results['significant_means'].head(2)

**Interaction scores** fields:
- **scores**: scores ranging from 0 to 100. The higher the score is,  the more specific the interaction is expected to be.

In [ ]:
cpdb_results['interaction_scores'].head(2)

**CellSign** fields:
- **interacting_pair**: Name of the interacting pairs.
- **partner A or B**: Identifier for the first interacting partner (A) or the second (B). It could be: UniProt (prefix simple:) or complex (prefix complex:)
- **gene A or B**: Gene identifier for the first interacting partner (A) or the second (B). The identifier will depend on the input user list.
- **Receptor A or B**: True if the first interacting partner (A) or the second (B) is annotated as a receptor in our database.
- **TF relationship**: a 1 denotes that the TF downstream the receptor is active.

In [ ]:
#cpdb_results['CellSign_active_interactions'].head(2)

**Deconvoluted** fields:
- **gene_name**: Gene identifier for one of the subunits that are participating in the interaction defined in “means.csv” file. The identifier will depend on the input of the user list.
- **uniprot**: UniProt identifier for one of the subunits that are participating in the interaction defined in “means.csv” file.
- **is_complex**: True if the subunit is part of a complex. Single if it is not, complex if it is.
- **protein_name**: Protein name for one of the subunits that are participating in the interaction defined in “means.csv” file.
- **complex_name**: Complex name if the subunit is part of a complex. Empty if not.
- **mean**: Mean expression of the corresponding gene in each cluster.

In [ ]:
cpdb_results['deconvoluted'].head(2)

### Explore CellphoneDB results
This module allows to filter CellphoneDB results by specifying: cell types pairs, genes or specific interactions. \

We can specify two list of cell types (`query_cell_type_1`and `query_cell_type_2`), the method will perform a pairwise comparison between the cell types defined in each list and will subset the interactions to those pairs of cells. Cell types within each list will not be paired. If we are interested in filtering interactions ocurring between a given cell to all the rest of cells in the dataset we can define `query_cell_type_1 = 'All'` and `query_cell_type_2 = ['cellA', 'cellB', ...]`. The argument `genes` allows the user to filter interactions in which a gene participates, the users can also define specific interactions based on the interaction name `query_interactions`.

> In **this example** we are going to filter interactions in which a subset of trophoblast interact with the PV cells, furthermore, we will select those interactions in which the gene TGFBR1 participates and a specific interaction named CSF1_CSF1R.

This method filters the rows and columns of the significant_means file; NaN value correspond to interacting pairs found not significant, non-NaN value correspond to those interacting pairs found relevant.

In [ ]:
celltype_a = "Cardiomyocyte"
celltype_b = "Endothelial Cell"
pair = f"{celltype_a}|{celltype_b}"


In [ ]:
sig_means = cpdb_results['significant_means'].copy()

# Keep only those interactions where the value is not NaN for our pair
sig_pairs = sig_means.loc[sig_means[pair].notna(), [
    "interacting_pair", "gene_a", "gene_b", "classification", pair
]]

# Sort by mean expression
sig_pairs = sig_pairs.sort_values(by=pair, ascending=False)


In [ ]:
sig_pairs.head(10)

In [ ]:
from cellphonedb.utils import search_utils

search_results = search_utils.search_analysis_results(
    query_cell_types_1 = ['All'],  # List of cells 1, will be paired to cells 2 (list or 'All').
    query_cell_types_2 = ['Cardiomyocyte'],     # List of cells 2, will be paired to cells 1 (list or 'All').
    query_genes = [ "VEGFB" ],                                       # filter interactions based on the genes participating (list).
    #query_interactions = ['CSF1_CSF1R'],                            # filter intereactions based on their name (list).
    significant_means = cpdb_results['significant_means'],          # significant_means file generated by CellphoneDB.
    deconvoluted = cpdb_results['deconvoluted'],                    # devonvoluted file generated by CellphoneDB.
    interaction_scores = cpdb_results['interaction_scores'],        # interaction score generated by CellphoneDB.
    query_minimum_score = 50,                                       # minimum score that an interaction must have to be filtered.
    separator = '|',                                                # separator (default: |) employed to split cells (cellA|cellB).
    long_format = True,                                             # converts the output into a wide table, removing non-significant interactions
    query_classifications = ['Signaling by Transforming growth factor']
)

search_results.head()

### Basic analysis and plotting
In this section of the tutorial we will make use of Kelvin's implementations ([kt-plots tutorial](https://ktplotspy.readthedocs.io/en/latest/index.html)) to perform some exploratory analysis of the results obtained from CellphoneDB. It is of great importance that the user is familiarized with the biological context of the data to obtain meaninful results from CellphoneDB results. 

In [ ]:
import os
import anndata as ad
import pandas as pd
import ktplotspy as kpy
import matplotlib.pyplot as plt
%matplotlib inline

Display as heatmap the number of significant interactions found by CellphoneDB between all of the cell pairs. Only cells found in the microenvironment will be represented in this plot.

In [ ]:
kpy.plot_cpdb_heatmap(pvals = cpdb_results['pvalues'],
                      degs_analysis = False,
                      figsize = (5, 5),
                      title = "Sum of significant interactions")

Here we are plotting the interactions between the PVs and the trophoblasts that are mediated by TGFB2 and CSF1R.

In [ ]:
kpy.plot_cpdb(
    adata = adata,
    cell_type1 = "PV MYH11|PV STEAP4|PV MMPP11",
    cell_type2 = "EVT_1|EVT_2|GC|iEVT|eEVT|VCT_CCC",
    means = cpdb_results['means'],
    pvals = cpdb_results['pvalues'],
    celltype_key = "cell_labels",
    genes = ["TGFB2", "CSF1R"],
    figsize = (10, 3),
    title = "Interactions between\nPV and trophoblast",
    max_size = 3,
    highlight_size = 0.75,
    degs_analysis = False,
    standard_scale = True,
    interaction_scores = cpdb_results['interaction_scores'],
    scale_alpha_by_interaction_scores = True
)

Interactions can also be plotted grouped by pathway.

In [ ]:
from plotnine import facet_wrap

p = kpy.plot_cpdb(
    adata = adata,
    cell_type1 = "PV MYH11",
    cell_type2 = "EVT_1|EVT_2|GC|iEVT|eEVT|VCT_CCC",
    means = cpdb_results['means'],
    pvals = cpdb_results['pvalues'],
    celltype_key = "cell_labels",
    genes = ["TGFB2", "CSF1R", "COL1A1"],
    figsize = (12, 8),
    title = "Interactions between PV and trophoblast\ns grouped by classification",
    max_size = 6,
    highlight_size = 0.75,
    degs_analysis = False,
    standard_scale = True,
)
p + facet_wrap("~ classification", ncol = 1)